<a href="https://colab.research.google.com/github/Tessellate-Imaging/monk_v1/blob/master/study_roadmaps/4_image_classification_zoo/Classifier%20-%20Invasive%20Species%20Monitoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## Install Monk


## Using pretrained model for Aerial Cactus Identification dataset


## Training a classifier from scratch

# Install Monk

  - git clone https://github.com/Tessellate-Imaging/monk_v1.git

  - cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt
        (Select the requirements file as per OS and CUDA version)


In [ ]:
! git clone https://github.com/Tessellate-Imaging/monk_v1.git

In [ ]:
# If using Colab install using the commands below
! cd monk_v1/installation/Misc && pip install -r requirements_colab.txt

# If using Kaggle uncomment the following command
#! cd monk_v1/installation/Misc && pip install -r requirements_kaggle.txt

# Select the requirements file as per OS and CUDA version when using a local system or cloud
#! cd monk_v1/installation/Linux && pip install -r requirements_cu9.txt



# Used trained classifier for demo

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Download trained weights

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1cRHTNHVEH2huvNBomgj8qSmsK_t_NM3J' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1cRHTNHVEH2huvNBomgj8qSmsK_t_NM3J" -O cls_invasive_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq cls_invasive_trained.zip 

In [ ]:
ls workspace/Project-Invasive-Species

In [ ]:
# Keras project
from keras_prototype import prototype

In [ ]:
# Load project in inference mode

gtf = prototype(verbose=1);
gtf.Prototype("Project-Invasive-Species", "Keras-resnet50_v2", eval_infer=True);

#Other trained models - uncomment 
#gtf.Prototype("Project-Invasive-Species", "Keras-resnet101_v2", eval_infer=True);
#gtf.Prototype("Project-Invasive-Species", "Keras-resnet152_v2", eval_infer=True);

In [ ]:
#Infer

In [ ]:
img_name = "workspace/test/1.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/2.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/3.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

In [ ]:
img_name = "workspace/test/4.jpg"
predictions = gtf.Infer(img_name=img_name);
from IPython.display import Image
Image(filename=img_name)

# Training custom classifier from scratch

## Dataset
    - Credits: https://www.kaggle.com/c/invasive-species-monitoring/data

## Download
    - sign up for the competition to download the data

In [ ]:
! pip install kaggle

In [ ]:
! kaggle competitions download -c invasive-species-monitoring

In [ ]:
! sudo apt-get install p7zip-full -y 

In [ ]:
! unzip -qq invasive-species-monitoring.zip -d invasive_dataset

In [ ]:
! 7z x invasive_dataset/train.7z -o./invasive_dataset/

In [ ]:
! 7z x invasive_dataset/test.7z -o./invasive_dataset/

In [ ]:
! unzip -qq invasive_dataset/train_labels.csv.zip -d invasive_dataset

In [ ]:
! unzip -qq invasive_dataset/sample_submission.csv.zip -d invasive_dataset

## Convert Labels

In [ ]:
import pandas as pd
df = pd.read_csv("invasive_dataset/train_labels.csv")

for i in range(len(df)):
    df["invasive"][i] = str(df["invasive"][i])
    df["name"][i] = str(df["name"][i]) + ".jpg"
    
df.to_csv("invasive_dataset/train_updated.csv", index=False)

## Training

In [ ]:
# Monk
import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Using mxnet-gluon backend 
#from gluon_prototype import prototype

# For pytorch backend
#from pytorch_prototype import prototype

# For Keras backend
from keras_prototype import prototype

In [ ]:
# Create Project and Experiment

gtf = prototype(verbose=1);
gtf.Prototype("Project-Invasive-Species", "Keras-resnet50_v2");

In [ ]:
gtf.Default(dataset_path="invasive_dataset/train/",
            path_to_csv="invasive_dataset/train_updated.csv",
            model_name="resnet50_v2", 
            freeze_base_network=False,
            num_epochs=2);

### How to change hyper parameters and models 
  - Docs - https://github.com/Tessellate-Imaging/monk_v1#4
  - Examples - https://github.com/Tessellate-Imaging/monk_v1/tree/master/study_roadmaps/1_getting_started_roadmap

In [ ]:
#Start Training
gtf.Train();

#Read the training summary generated once you run the cell and training is completed

## Testing on a new dataset for submission

In [ ]:
# Import monk

import os
import sys
sys.path.append("monk_v1/monk/");

In [ ]:
# Using mxnet-gluon backend 
#from gluon_prototype import prototype

# For pytorch backend
#from pytorch_prototype import prototype

# For Keras backend
from keras_prototype import prototype

In [ ]:
# Create Project and Experiment

gtf = prototype(verbose=1);
gtf.Prototype("Project-Invasive-Species", "Keras-resnet50_v2", eval_infer=True);

In [ ]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from scipy.special import softmax
df = pd.read_csv("invasive_dataset/sample_submission.csv")

In [ ]:
df.columns

In [ ]:
for i in tqdm(range(len(df))):
    img_name = "invasive_dataset/test/" + str(df["name"][i]) + ".jpg";
    
    #Invoking Monk's nferencing engine inside a loop
    predictions = gtf.Infer(img_name=img_name, return_raw=False);
    out = predictions["predicted_class"]
    
    df["invasive"][i] = int(out);


In [ ]:
df.to_csv("submission.csv", index=False);